# <font color=#00988c> AudioTact: Texture decoding intra modality and cross modalities

Created on Wed Jul 10 16:06:11 2019  
Last update: Wed Feb 10 2021  
*@author: Caroline Landelle (caroline.landelle@mcgill.ca // landelle.caroline@gmail.com )*

**dataset:** AudioTact Young  
**toolboxes required:** Nilearn (python)  
**Description:** This notebook provides code for AudioTact dataset. We used searchlight-based multi-voxel pattern analysis (MVPA) to elucidate whether the patterns of response to rough or smooth stimuli can be differentiated within a modality and whether the textured stimuli could be decoded in common brain regions regardless of their sensory origin.

**Task**
* Roughness estimation in unimodal (Audio, A or Haptic, H) or congruent bimodal (Audio-Haptic, AH) conditions
* 5 successive runs, each consisting of 4 blocks of 4 conditions (A; H; AH; Rest) 


____________________________________________________

**Input** :
   > - fMRI datas : betasXXX.nii  (u-img + Tapas + 1 regressor for each trial) unsmoothed. Separate regressors for each trial were modelled by convolving a boxcar function describing the timing of stimulus events with the canonical hemodynamic response function (HRF) used in SPM. Data were normalized to native space (T1w)    
   > - Anat : u-anat.nii (coregister)    
   > - mask.nii : anatomical (WM+GM+LCR) mask  
   > - Design.csv :  
   > 1 beta for each trial of interest => 12x3Modalityx2Texture  
   > 1 beta for all trials of a none interest condition => 1 for Rest, 1 for Cue, 1 for Response)  
   
**Output** :
   > - accMap_XXX.nii : Accuracy maps  
   >  *These analyses led to classification  accuracy  at  each  voxel  for each  participant  and  the  resulting  whole  brain accuracy maps*  
   > - NormAccMap_XXX.nii : t-maps  
   >  *To perform  the  group  level  analysis,  chance  level accuracy  map  (0.5)  was  subtracted  from  each participant’s classification  accuracy  map.*

## <font color=#00988c> I. Imports & Variables initialization </font> 
### <font color=#00988c> I.1. Imports </font>

In [39]:
import pandas as pd
import numpy as np
import os

# nilearn
import nilearn.decoding
from nilearn.image import new_img_like, load_img, index_img

#sklearn
from sklearn.model_selection import LeaveOneGroupOut, LeavePGroupsOut

### <font color=#00988c>  I.2 Define directories and initialize variables </font>
Directories must be modified if the analyses are not process on CL Macbook

In [40]:
# A. Main directories: _____________________________________________________
Directories={'DataDir':'/Users/carolinelandelle/Research_Project/Audio-Tact_fMRI/Datas_Young/',
             'OnsetsDir':'/Ana_CL1/Ana_MVPA/Onsets',
             'BetaDir':'/BetaTrialDesign_',
             'GLMDir':'/Ana_CL1/Ana_MVPA/Betas_trials/',
             'outputDir':'/Ana_CL1/Ana_MVPA/Results/LTO/Cross-Modal-Splits/',
             'AnatFile':'/anat/Anat_Masked.nii',
             'MaskFile':'anat/explicitMask_Coreg_03.nii', #T1w mask in the native space,
            }

# Participant to be processed in next parts of the notebook:
Directories['list_subjects']=['sub-2'] #  ['sub-test1','sub-1','sub-2','sub-5','sub-6','sub-7','sub-9','sub-10','sub-11','sub-12','sub-13','sub-17','sub-19','sub-20','sub-22','sub-23']'sub-24','sub-25','sub-26','sub-35','sub-test1']

# B. Design files information: _____________________________________________________
Datafile={'Subjects':'Subject', # name of the column that define the subject number
    'Class':'Texture',# name of the colum that define the different class of decoding
    'Class_Levels':['rough','smooth'], # different class levels
    'Modality':'Modality',#name of the colum that define the different Modalities
    'Modalities':['A','H','AH'], # different modalities
    'Runs':'Run'} #name of the colum that define the different runs

#Check the variables
print('Main directories: ____________________________________________________') 
print(Directories)
print('                                 ') 
print('Name of column datafile: _______________________________________________') 
print(Datafile)

Main directories: ____________________________________________________
{'DataDir': '/Users/carolinelandelle/Research_Project/Audio-Tact_fMRI/Datas_Young/', 'OnsetsDir': '/Ana_CL1/Ana_MVPA/Onsets', 'BetaDir': '/BetaTrialDesign_', 'GLMDir': '/Ana_CL1/Ana_MVPA/Betas_trials/', 'outputDir': '/Ana_CL1/Ana_MVPA/Results/LTO/Cross-Modal-Splits/', 'AnatFile': '/anat/Anat_Masked.nii', 'MaskFile': 'anat/explicitMask_Coreg_03.nii', 'list_subjects': ['sub-2']}
                                 
Name of column datafile: _______________________________________________
{'Subjects': 'Subject', 'Class': 'Texture', 'Class_Levels': ['rough', 'smooth'], 'Modality': 'Modality', 'Modalities': ['A', 'H', 'AH'], 'Runs': 'Run'}


### <font color=#00988c>  I.3 Individual files and directiories </font>

In [41]:
# A. Define empty dictionnaries:
subDir={} # individual input directories
csvfileMVPA={} # Individual design files
mask_img={} # brain mask image (wm + gm + csf)
anat_File={} # structural image
Beta={} # beta images
design_file={}

for sub in Directories['list_subjects']:
# B. Define indivudal files:
    #d = pd.DataFrame()
    subDir[sub]= Directories['DataDir'] +  sub + '/'
    csvfileMVPA[sub] = subDir[sub] + Directories['OnsetsDir'] + Directories['BetaDir'] + sub + '.csv'
    design_file[sub] = pd.read_csv(csvfileMVPA[sub], sep = ",") # read cvs doc
    
    anat_File[sub]=load_img(subDir[sub] + Directories['AnatFile'])# load anat
    mask_img[sub]=load_img(subDir[sub] + Directories['MaskFile']) # load mask
    
    print('Anat file:' + subDir[sub] + Directories['AnatFile'])
    print('Mask file:' + subDir[sub] + Directories['MaskFile'])
    print(' ')
    
    Beta[sub] = design_file[sub]['Beta'][(design_file[sub]['Subject']==sub)] #Beta img
    print('Reading beta maps: ____________________________') # could take time
    print(Beta[sub][0] + ' '+ Beta[sub][1] + ' ...')
    for i in Beta[sub].index:
        Beta[sub][i]= os.path.join(subDir[sub] + Directories['GLMDir'], Beta[sub][i] + ".nii") # Beta images
        
        
    
    
    
    print(' ')
    print('Design file : ____________________________________________')
    print(design_file[sub]) # exemple of design file



Anat file:/Users/carolinelandelle/Research_Project/Audio-Tact_fMRI/Datas_Young/sub-2//anat/Anat_Masked.nii
Mask file:/Users/carolinelandelle/Research_Project/Audio-Tact_fMRI/Datas_Young/sub-2/anat/explicitMask_Coreg_03.nii
 
Reading beta maps: ____________________________
beta_0001 beta_0002 ...
 
Design file : ____________________________________________
    Subject  Run     Condition  Modality Texture       Beta
0     sub-2    1     A_rough_1         A   rough  beta_0001
1     sub-2    1     A_rough_2         A   rough  beta_0002
2     sub-2    1     A_rough_3         A   rough  beta_0003
3     sub-2    1     A_rough_4         A   rough  beta_0004
4     sub-2    1     A_rough_5         A   rough  beta_0005
..      ...  ...           ...       ...     ...        ...
370   sub-2    5  AH_smooth_11        AH  smooth  beta_0563
371   sub-2    5  AH_smooth_12        AH  smooth  beta_0564
372   sub-2    5          Rest      Rest     NaN  beta_0565
373   sub-2    5           Cue       Cue  

### <font color=#00988c>  I.4 Experimental design </font>

In [42]:
Class={}
Class_mask={}
modality={}

## number of run is the same for all participants
run_numbers=np.unique(design_file[sub]['Run'])
nb_runs=np.max(run_numbers)
print('Runs: ' + str(run_numbers))


for sub in Directories['list_subjects']:# C. Define experimental design _____________________________________
    Class[sub] = design_file[sub][Datafile['Class']][design_file[sub][Datafile['Subjects']]==sub]
    Class_mask[sub] = Class[sub].isin(Datafile['Class_Levels']) # boolean serie (T or F)
    modality[sub]= design_file[sub]['Modality'][design_file[sub][Datafile['Subjects']]==sub]
    print(' ')
    print('Select the classes : ____________________________')
    print(Class[sub]) # exemple of design file
    print(' ')
    print('Classes T/F : ____________________________')
    print(Class_mask[sub])
    print(' ')
    print('Select the modalities : ____________________________')
    print(modality[sub])

Runs: [1 2 3 4 5]
 
Select the classes : ____________________________
0       rough
1       rough
2       rough
3       rough
4       rough
        ...  
370    smooth
371    smooth
372       NaN
373       NaN
374       NaN
Name: Texture, Length: 375, dtype: object
 
Classes T/F : ____________________________
0       True
1       True
2       True
3       True
4       True
       ...  
370     True
371     True
372    False
373    False
374    False
Name: Texture, Length: 375, dtype: bool
 
Select the modalities : ____________________________
0             A
1             A
2             A
3             A
4             A
         ...   
370          AH
371          AH
372        Rest
373         Cue
374    Response
Name: Modality, Length: 375, dtype: object


## <font color=#00988c> II. Define the analysis </font> 

In [43]:
# II.1 Searchlight parameters : ______________________________________________________
#LeavePGroupsOut()/LeaveOneGroupOut(): split data according to predifined groups. 
# 'groups' are the different runs of acquisition and thus allow a cross validation with respect to the different runs.
Searchlight={'radius':10, # radius in mm of the sphere
            'classifier':'svc', #classifier = svc : support vector machine
            'n_groups':2, # number of test split, 1 for leaveOneOut, 2 for leaveTwoOut
            'chance_level':0.5} # The change level is set at 50% (0.5) for two classes

cv=LeavePGroupsOut(n_groups=Searchlight['n_groups']) #LeaveOneGroupOut() Number of groups (p) to leave out in the test split.

# II.2 Choose the decoding mode : Decoding cross modality 'Yes' or not 'No'
Cross_Modality='Yes'#'No'#  Decoding cross modality 'Yes' or not 'No'
CrossDecoding=['A_H'] # Change only the letter after _  (used the order as train_test)

if Cross_Modality=='No':
    print('One Modality analyse')

elif Cross_Modality=='Yes':
    print('Cross Modality analyse')
else:
    print('no analyse definition')



Cross Modality analyse


## <font color=#00988c> II. Define train and test trials </font> 

In [44]:
mask_train={} 
mask_train={} 
split_run={}
split_mod={}
split_class={}
split_mask_run={}
split_mask_mod={}
split_mask_class={}

for sub in Directories['list_subjects']:
    for mod in CrossDecoding:
        # A. Select train and test trials______________________________________________________
        if Cross_Modality=='No': #e.g [A_A] or [H_H] or [AH_AH]
            if mod.split('_')[0] == Datafile['Modalities'][0]:
                mask_train[sub]= modality[sub].isin([Datafile['Modalities'][0]]) # boolean serie (T or F) => T: Modality to be tested
            elif mod.split('_')[0] ==Datafile['Modalities'][1]:
                mask_train[sub]=modality[sub].isin([Datafile['Modalities'][1]])  # boolean serie (T or F) => T: Modality to be tested
            elif mod.split('_')[0] ==Datafile['Modalities'][2]:
                mask_train[sub]= modality[sub].isin([Datafile['Modalities'][2]])  # boolean serie (T or F) => T: Modality to be tested

        elif Cross_Modality=='Yes': #e.g [A_H]
            Mod2Train, Mod2Test = mod.split('_')# order of train and test in not important for this step
            mask_train[sub]= modality[sub].isin([Mod2Train,Mod2Test]) # boolean serie (T or F) => T: Modality to be t
        
        print('select the Modality for train trials: __________________')
        print(mask_train[sub])
        
        # B. Select in the design runs, modality and classes ______________________________________________________
        split_run[sub] = design_file[sub][Datafile['Runs']][(design_file[sub][Datafile['Subjects']]==sub)] # split data between Runs  ### not used here after all
        split_mod[sub] = design_file[sub][Datafile['Modality']][(design_file[sub][Datafile['Subjects']]==sub)] # split data between modality
        split_class[sub] = design_file[sub][Datafile['Class']][(design_file[sub][Datafile['Subjects']]==sub)] # split data between class
        print(' ')
        print('Exemple: select read the column __________________')
        print(split_class[sub])
        
        #C. Keep only the condition of interest
        
        stim_train_mask = (Class_mask[sub]) & (mask_train[sub])
        # Select in these column only the trial of interest (ex: True for trials corresponding to the mod tested)
        split_mask_run[sub]=split_run[sub][mask_train[sub]]
        split_mask_mod[sub]=split_mod[sub][mask_train[sub]]
        split_mask_class[sub]=split_class[sub][mask_train[sub]]
        
        # Reindex the raw number of the variables
        split_mask_run[sub].index=range(len(split_mask_run[sub]))
        split_mask_mod[sub].index=range(len(split_mask_mod[sub]))
        split_mask_class[sub].index=range(len(split_mask_class[sub]))
        
        print(' ')
        print('Exemple: select only the class of interest __________________')
        print(split_mask_class[sub])
        

select the Modality for train trials: __________________
0       True
1       True
2       True
3       True
4       True
       ...  
370    False
371    False
372    False
373    False
374    False
Name: Modality, Length: 375, dtype: bool
 
Exemple: select read the column __________________
0       rough
1       rough
2       rough
3       rough
4       rough
        ...  
370    smooth
371    smooth
372       NaN
373       NaN
374       NaN
Name: Texture, Length: 375, dtype: object
 
Exemple: select only the class of interest __________________
0       rough
1       rough
2       rough
3       rough
4       rough
        ...  
235    smooth
236    smooth
237    smooth
238    smooth
239    smooth
Name: Texture, Length: 240, dtype: object


## <font color=#00988c> III. Run searchlight analysis

In [ ]:
n_splits={} # total number of splits 
split_ind={} # split number
train_inds={} # trial number for training, lenght= number of run to be trained x number of trial tested in each run
test_inds={} # trial number for testing, lenght= number of run to be tested x number of trial trained in each run

trials_runtest={}
trial_test={}
trial_test_id={}

trials_runtrain={}
trial_train={}
trial_train_id={}

single_split={}
X_fmri_img={}

for sub in Directories['list_subjects']:
# III.1 Check the splits that are being computed___________________________________
    print('*****************************************************')
    print('Splits for subject: ' + sub)
    n_splits[sub]=cv.get_n_splits(groups=split_mask_run[sub])# number of split in train and test.
    print(' ')
    print('Total number of splits: ' + str(n_splits[sub]))
    
    for split_ind[sub], (train_inds[sub],test_inds[sub]) in enumerate(cv.split(split_mask_run[sub],split_mask_run[sub],split_mask_run[sub])):
        print(" ")
        print("Split number " + str(split_ind[sub]+1) + ' __________________________') # split number (+1 beacuse it start at 0)
        print("Run Train :")
        for i in range(0,(nb_runs-Searchlight['n_groups'])): # print the X run number for the training
            print(str(split_mask_run[sub][train_inds[sub][int(int(i)*len(split_mask_class[sub])/(split_mask_run[sub][-1:]))]]))
        print("Run Test :")
        for j in range(0,Searchlight['n_groups']): # print the X run number for the testing (n_groups tested = number of run left out)
            print(str(split_mask_run[sub][test_inds[sub][int(j+1)*int(int(len(test_inds[sub])/Searchlight['n_groups'])/2)]])) ## number here is nb of test trial per split (so if one run and one modality : 24)

# III.2 Retrieve the real indices to be used in the test and train sets ____________________________________________
        trials_runtest[sub]=split_mask_mod[sub][test_inds[sub]] # all trials in the test runs
        trial_test[sub]=trials_runtest[sub].isin([Mod2Test]) # trials in the test runs corresponding to the modality to be tested
        trial_test_id[sub]=np.array(trial_test[sub][trial_test[sub]].index) # trials indiced for testing (ie 12 trials*2 level*2runs, here=48)
            
        trials_runtrain[sub]=split_mask_mod[sub][train_inds[sub]]# all trials in the train runs
        trial_train[sub]=trials_runtrain[sub].isin([Mod2Train]) # trials in the test runs corresponding to the modality to be trained
        trial_train_id[sub]=np.array(trial_train[sub][trial_train[sub]].index) # trials indiced for testing (ie 12 trials*two level*3runs, here=72)
            
        class_labels = np.unique(split_mask_class[sub])
        #print(" ")
        #print("Index of the train trials: ")
        #print(trial_train_id[sub])
        #print("Index of the test trials: ")
        #print(trial_test_id[sub])
    
        single_split[sub] = [(trial_train_id[sub],trial_test_id[sub])]
            
        X_fmri_img[sub] = index_img(Beta[sub], mask_train[sub])
        #X_fmri_img.shape #should = to the nb of trial tested
                     
                     
# III.3 Searchlight definition ____________________________________________
        searchlight = nilearn.decoding.SearchLight(mask_img[sub],
                                                   radius=Searchlight['radius'],
                                                   estimator=Searchlight['classifier'],
                                                   n_jobs=1,
                                                   scoring = None, # The scoring strategy to use.If callable, takes as arguments the fitted estimator, the test data (X_test) and the test target (y_test) if y is not None.
                                                   verbose=1,
                                                   cv=single_split[sub])
            
        print("...mapping the data (this takes a long time) and fitting the model in each sphere")
        
# III.4 Run Searchlight ____________________________________________
        searchlight.fit(X_fmri_img[sub], split_mask_class[sub])
                    # Save Searchlight
        single_split_nii = new_img_like(mask_img[sub],searchlight.scores_ - Searchlight['chance_level'])

        Filename = 'Train_'+ Mod2Train + '_'  + 'Test_'+ Mod2Test + '_' + str(Searchlight['radius']) +'mm' + '_Split_0' + str(split_ind[sub]+1)
           
        single_split_path = os.path.join(subDir[sub] + Directories['outputDir'] + "Searchlight_" + Filename)
        print('Saving score map for searchlight : '+ mod + ' - Split nb ' + str(split_ind[sub]+1))
        single_split_nii.to_filename(single_split_path)

*****************************************************
Splits for subject: sub-2
 
Total number of splits: 10
 
Split number 1 __________________________
Run Train :
3
4
5
Run Test :
1
2
...mapping the data (this takes a long time) and fitting the model in each sphere
